![NeZha1](https://img-blog.csdnimg.cn/20210708204825728.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![NeZha2](https://img-blog.csdnimg.cn/20210708205133810.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![全词编码](https://img-blog.csdnimg.cn/2021070821003392.png)

In [ ]:
! pip install transformers==4.0.1

In [ ]:
import torch
import random
import numpy as np
config = {
    'train_file_path': 'data/data86671/train.csv',
    'test_file_path': 'data/data86671/test.csv',
    'train_val_ratio': 0.1,

    # ------------- 新加代码 --------------- #

    # ------------- 新加代码 ----------------#
    
    'batch_size': 16,   # 我们要使用 BERT , 所以 batch_size 小一点
    'num_epochs': 2,
    'learning_rate': 2e-5,
    'logging_step': 500,
    'seed': 2021}

config['device']='cuda' if torch.cuda.is_available() else 'cpu'

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(config['seed'])

In [ ]:
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
def read_data(config, tokenizer, mode='train'):
    data_df = pd.read_csv(config[f'{mode}_file_path'], sep=',')
    if mode == 'train':
        X_train, y_train = defaultdict(list), []
        X_val, y_val = defaultdict(list), []
        num_val = int(len(data_df) * config['train_val_ratio'])
    else:
        X_test, y_test = defaultdict(list), []

    for i, row in tqdm(data_df.iterrows(), desc=f'preprocess {mode} data', total=len(data_df)):
        label=row[1] if mode == 'train' else 0
        sentence = row[-1]

        # ---------------------- 新加代码 ---------------------- #
   
        # ---------------------- 新加代码 ---------------------- #

        if mode == 'train':
            if i < num_val:
                X_val['inputs_ids'].append(inputs['input_ids'])
                y_val.append(label)
                
                # ---------------------- 新加代码 ---------------------- #
   
                # ---------------------- 新加代码 ---------------------- #
                                
            else:
                X_train['inputs_ids'].append(inputs['input_ids'])
                y_train.append(label)

                # ---------------------- 新加代码 ---------------------- #
   
                # ---------------------- 新加代码 ---------------------- #

        else:
            X_test['inputs_ids'].append(inputs['input_ids'])
            y_test.append(label) 
            # ---------------------- 新加代码 ---------------------- # 

            # ---------------------- 新加代码 ---------------------- #
            

    if mode == 'train':
        label2id = {label: i for i, label in enumerate(np.unique(y_train))} 
        id2label = {i: label for label, i in label2id.items()} 
        y_train = torch.tensor([label2id[i] for i in y_train], dtype=torch.long)  
        y_val = torch.tensor([label2id[i] for i in y_val], dtype=torch.long)  
        return X_train, y_train, X_val, y_val, label2id, id2label
        
    else:
        y_test = torch.tensor(y_test, dtype=torch.long)
        return X_test, y_test

In [ ]:
from torch.utils.data import Dataset
class TNEWSData(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y

    def __getitem__(self, idx):
        return {
            'inputs_ids' : self.x['inputs_ids'][idx],
            'label' : self.y[idx],
            # ---------------------- 新加代码 ---------------------- #

            # ---------------------- 新加代码 ---------------------- #
        }
    
    def __len__(self):
        return self.y.size(0)

In [ ]:
def collate_fn(examples):
    input_ids_list = []
    labels = []
    # ---------------------- 新加代码 ---------------------- #

    # ---------------------- 新加代码 ---------------------- #

    for example in examples:
        input_ids_list.append(example['inputs_ids'])
        labels.append(example['label'])
        # ---------------------- 新加代码 ---------------------- #

        # ---------------------- 新加代码 ---------------------- #
    
    max_length = max(len(input_ids) for input_ids in input_ids_list)
    input_ids_tensor = torch.zeros((len(labels), max_length), dtype=torch.long)

    # ---------------------- 新加代码 ---------------------- #

    # ---------------------- 新加代码 ---------------------- #

    for i, input_ids in enumerate(input_ids_list):
        input_ids_tensor[i, :len(input_ids)] = torch.tensor(input_ids, dtype=torch.long)
        # ---------------------- 新加代码 ---------------------- #

        # ---------------------- 新加代码 ---------------------- #
    
    return{
        'input_ids' : input_ids_tensor,
        'labels' : torch.tensor(labels, dtype=torch.long),
        # ---------------------- 新加代码 ---------------------- #

        # ---------------------- 新加代码 ---------------------- #
    }

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import DataLoader
def build_dataloader(config):
    # ---------------------- 新加代码 ---------------------- #

    # ---------------------- 新加代码 ---------------------- #
    X_train, y_train, X_val, y_val, label2id, id2label = read_data(config, tokenizer, mode='train')
    X_test, y_test = read_data(config, tokenizer, mode='test')

    train_dataset = TNEWSData(X_train, y_train)
    val_dataset = TNEWSData(X_val, y_val)
    test_dataset = TNEWSData(X_test, y_test)

    train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], num_workers=4, shuffle=True, collate_fn=collate_fn)
    val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], num_workers=4, shuffle=False, collate_fn=collate_fn)
    test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], num_workers=4, shuffle=False, collate_fn=collate_fn)

    return train_dataloader, val_dataloader, test_dataloader, id2label

In [ ]:
train_dataloader, val_dataloader, test_dataloader, id2label = build_dataloader(config)

In [ ]:
for batch in train_dataloader:
    print(len(batch['input_ids']))
    print(batch)
    break

In [ ]:
from sklearn.metrics import f1_score
def evaluation(config, model, val_dataloader):
    model.eval()
    preds = []
    labels = []
    val_loss = 0.
    val_iterator = tqdm(val_dataloader, desc='Evaluation', total=len(val_dataloader))

    with torch.no_grad():
        for batch in val_iterator:
            labels.append(batch['labels'])
            batch = {item: value.to(config['device']) for item, value in batch.items()}
            loss, logits = model(**batch)[:2]

            val_loss += loss.item()
            preds.append(logits.argmax(dim=-1).detach().cpu())

    avg_val_loss = val_loss / len(val_dataloader)
    labels = torch.cat(labels, dim=0).numpy()
    preds = torch.cat(preds, dim=0).numpy()
    f1 = f1_score(labels, preds, average='macro')
    return avg_val_loss, f1